In [1]:
import pyspark
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt
import numpy as np
import h5py
import pandas as pd
import glob
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType

In [2]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.130:7077") \
        .appName("John")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .config("spark.hadoop.fs.defaultFS", "hdfs://192.168.2.130:9000")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 15:54:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/07 15:54:16 WARN Utils: Service 'sparkDriver' could not bind on port 9999. Attempting port 10000.
24/03/07 15:54:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/07 15:54:17 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 10005. Attempting port 10006.


In [3]:
direct_path = "hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5"

df = spark_session.read.format("binaryFile").load(direct_path)
h5_file_paths = df.select("path").rdd.map(lambda row: row.path).collect()
print("Found .h5 files:", h5_file_paths)

24/03/07 15:54:38 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/07 15:54:53 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/07 15:55:08 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/07 15:55:23 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/07 15:55:38 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/07 15:55:53 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

Found .h5 files: ['hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5']


In [20]:
# store = pd.HDFStore(direct_path, 'w')
# pdf = pd.read_hdf(direct_path)

dhf = pd.read_hdf(direct_path)

FileNotFoundError: File hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5 does not exist

In [18]:
print(store.keys)
store.select('direct_path') 

<bound method HDFStore.keys of <class 'pandas.io.pytables.HDFStore'>
File path: direct_path
>


KeyError: 'No object named file path in the file'

In [5]:
sc = spark_session.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
file_system = sc._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_conf)

def list_h5_files(path, fs=file_system):
    files = []
    status = fs.listStatus(sc._jvm.org.apache.hadoop.fs.Path(path))
    for file_status in status:
        file_path = file_status.getPath()
        if file_status.isDirectory():
            files.extend(list_h5_files(file_path.toString()))
        elif file_path.toString().endswith(".h5"):
            files.append(file_path.toString())
    return files

base_directory = "hdfs://192.168.2.130:9000/user/MillionSongSubset"
h5_files = list_h5_files(base_directory)
print(f"Found {len(h5_files)} .h5 files.")

Found 10000 .h5 files.


In [6]:
print(h5_files[:3])

['hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5', 'hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAABD128F429CF47.h5', 'hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAADZ128F9348C2E.h5']


In [7]:
def read_h5_to_row(filename):
    """
    Read a single HDF5 file and return a tuple representing a row of data.
    """
    try:
        with h5py.File(filename, 'r') as file:
            return (
                float(file['analysis']['songs'][0][23]),
                int(file['musicbrainz']['songs'][0][1]),
            )
    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        return None

In [8]:
#file_paths_rdd = sc.parallelize(h5_files[:10])
# song_data_rdd = spark_session.sparkContext.parallelize(h5_files[:10]).map(read_h5_to_row)
read_h5_to_row(direct_path)

Error processing file hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5: [Errno 2] Unable to synchronously open file (unable to open file: name = 'hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [9]:
# def extract_data_from_h5_in_memory(file_content):
#     data = {}
#     try:
#         with h5py.File(io.BytesIO(file_content), 'r') as h5_file:
#             data['loudness'] = h5_file['/analysis/songs'][0][23]
#             data['year'] = h5_file['/analysis/songs'][0][1]
#     except Exception as e:
#         data = {'error': str(e)}
#     return data


In [10]:
# import pyarrow

# def read_h5_file_from_hdfs(hdfs_path, hdfs_host='default', hdfs_port=9000):
#     # Connect to HDFS
#     hdfs = pyarrow.HadoopFileSystem(host=hdfs_host, port=hdfs_port)
    
#     # Read the file into memory
#     with hdfs.open(hdfs_path, 'rb') as f:
#         file_content = f.read()
    
#     return file_content


In [11]:
# data = read_h5_file_from_hdfs(direct_path)

In [12]:
# def process_file(file_path):
#     file_content = read_h5_file_from_hdfs(file_path)
#     return extract_data_from_h5_in_memory(file_content)

# # Distribute the work
# file_paths_rdd = sc.parallelize(h5_files[:10])
# extracted_data_rdd = file_paths_rdd.map(lambda file_path: process_file(file_path))

# # # Collect the results
# # extracted_data = extracted_data_rdd.collect()


In [13]:
# for item in extracted_data_rdd.take(5):
#     print(item)

In [14]:
spark_session.stop()